In [1]:
import numpy as np
from tqdm import tqdm

from torch.utils.data.sampler import SubsetRandomSampler

%run models.ipynb

In [19]:
# Preparing the splits
def split_dataset(train_set, n_splits):
    indices = list(range(len(train_set)))
    np.random.shuffle(indices)
    split_size = int(np.floor(1/n_splits * len(train_set)))
    datasets = []
    dataloaders = []
    
    for i in range(n_splits):
        start = i * split_size
        end = start + split_size
        curr_indices = indices[start:end]
        
        # Get valid indicies and train indicies from the curr
        np.random.shuffle(curr_indices)
        # to get 20% of the train set
        split = int(np.floor(0.2 * len(curr_indices)))
        
        valid_sample = SubsetRandomSampler(curr_indices[:split])
        train_sample = SubsetRandomSampler(curr_indices[split:])
        datasets.append((train_sample, valid_sample))

        train_loader = torch.utils.data.DataLoader(train_set, sampler=train_sample, batch_size=64, generator=torch.Generator(device))
        valid_loader = torch.utils.data.DataLoader(train_set, sampler=valid_sample, batch_size=64, generator=torch.Generator(device))
        dataloaders.append((train_loader, valid_loader))
        
    return datasets, dataloaders

Split 0: Train sample len: 4800, Valid sample len: 1200
Split 1: Train sample len: 4800, Valid sample len: 1200
Split 2: Train sample len: 4800, Valid sample len: 1200
Split 3: Train sample len: 4800, Valid sample len: 1200
Split 4: Train sample len: 4800, Valid sample len: 1200
Split 5: Train sample len: 4800, Valid sample len: 1200
Split 6: Train sample len: 4800, Valid sample len: 1200
Split 7: Train sample len: 4800, Valid sample len: 1200
Split 8: Train sample len: 4800, Valid sample len: 1200
Split 9: Train sample len: 4800, Valid sample len: 1200


In [ ]:
def train_split_models(train_set, epochs=10, n_splits=10):
    models = [FashionCNN().to(device) for i in range(n_splits)]
    datasets, dataloaders = split_dataset(train_set, n_splits)
    
    model_avg_train_loss = []
    model_avg_valid_loss = []
    model_avg_valid_acc = []

    # initialize the ensemble model
    for idx, model in enumerate(models):
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.005)
        criterion = nn.CrossEntropyLoss()
        train_loss, valid_loss, valid_acc = train_model(
            model, dataloaders[0][0], dataloaders[0][1],
            optimizer, criterion, epochs
        )
        model_avg_train_loss.append(train_loss)
        model_avg_valid_loss.append(valid_loss)
        model_avg_valid_acc.append(valid_acc)
    
    avg_train_loss = sum(model_avg_train_loss)/n_splits
    avg_valid_loss = sum(model_avg_valid_loss)/n_splits
    avg_valid_acc = sum(model_avg_valid_acc)/n_splits
    return avg_train_loss, avg_valid_loss, avg_valid_acc